In [ ]:
from __init__ import *

In [ ]:
w_df_, h_df_ = read_factorization(
    "../output/matrix_factorization/", SETTING["model_mode"], SETTING["model_data_"]
)

In [ ]:
distance_function = kraft.information.get_icd

In [ ]:
size = 640

layout = {"height": size, "width": size}

In [ ]:
factor_label_ = h_df_[0].index.to_numpy()

factor_name = "Factor"

In [ ]:
w_node_x_node = []

for index, df in enumerate(w_df_):

    matrix = df.to_numpy().T

    node_x_node = kraft.array.function_on_2_2d_array(matrix, matrix, distance_function)

    kraft.plot.plot_heat_map(
        node_x_node,
        factor_label_,
        factor_label_,
        factor_name,
        factor_name,
        layout={"title": {"text": "W {} Distance".format(index)}, **layout},
    )

    w_node_x_node.append(node_x_node)

In [ ]:
h_node_x_node = []

for index, df in enumerate(h_df_):

    matrix = df.to_numpy()

    node_x_node = kraft.array.function_on_2_2d_array(
        matrix,
        matrix,
        distance_function,
    )

    kraft.plot.plot_heat_map(
        node_x_node,
        factor_label_,
        factor_label_,
        factor_name,
        factor_name,
        layout={"title": {"text": "H {} Distance".format(index)}, **layout},
    )

    h_node_x_node.append(node_x_node)

In [ ]:
node_x_node = np.median(np.asarray(w_node_x_node + h_node_x_node), axis=0)

kraft.plot.plot_heat_map(
    node_x_node,
    factor_label_,
    factor_label_,
    factor_name,
    factor_name,
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
node_marker_size = 24

if 1 < len(SETTING["resolution_"]):

    r_ = np.asarray(
        tuple(int(label.split(maxsplit=1)[1][0]) for label in factor_label_)
    )

    node_marker_size *= 2 ** (1 - kraft.array.normalize(r_, "0-1"))

In [ ]:
for w_h_symbol in ("w", "h"):

    if w_h_symbol == "w":

        point_x_node = pd.concat(w_df_)

    else:

        point_x_node = pd.concat(h_df_, axis=1).T

    directory_path = "../output/gps_map/{}/".format(w_h_symbol)

    kraft.path.make(directory_path)

    gps_map = kraft.gps_map.GPSMap(
        factor_name,
        factor_label_,
        node_x_node,
        point_x_node.index.name,
        point_x_node.index.to_numpy(),
        point_x_node.to_numpy() ** SETTING["{}_gps_map_node_power".format(w_h_symbol)],
        node_marker_size=node_marker_size,
    )

    gps_map.set_group("closest_node")

    kraft.gps_map.write(
        "{}gps_map.pickle.gz".format(directory_path, w_h_symbol), gps_map
    )

    gps_map.plot(
        point_trace={
            "marker": {"size": SETTING["{}_gps_map_point_size".format(w_h_symbol)]}
        },
        file_path="{}/gps_map.html".format(directory_path),
    )